Modified: Jul 18, 2019
# Gradient field visualization
- Correctly ground numpy array to visualize in the Cartesian coordinate system


In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, time
import numpy as np
import scipy as sp
from scipy.signal import correlate2d
import pandas as pd
    
from pathlib import Path
from pprint import pprint as pp

from sklearn.externals import joblib
import pdb

import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
# Don't generate bytecode
sys.dont_write_bytecode = True

In [ ]:
import holoviews as hv
import xarray as xr

from holoviews import opts, dim
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.streams import Stream, param
from holoviews import streams
import geoviews as gv
import geoviews.feature as gf
from geoviews import tile_sources as gvts

import panel as pn
import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cf

hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'
pn.extension()

In [ ]:
# Add the utils directory to the search path
UTILS_DIR = Path('../utils').absolute()
assert UTILS_DIR.exists()
if str(UTILS_DIR) not in sys.path:
    sys.path.insert(0, str(UTILS_DIR))
    print(f"Added {str(UTILS_DIR)} to sys.path")

# pp(sys.path)
    

In [ ]:
from utils import get_mro as mro, nprint
import utils as u
from vector import Vector as vec

### Set visualization options

In [ ]:
%opts Image [colorbar=True, active_tools=['wheel_zoom'], tools=['hover']] Curve [tools=['hover'], active_tools=['wheel_zoom']] RGB [active_tools=['wheel_zoom'], tools=['hover']]

In [ ]:
H, W = 300,300
img_opts = opts.Image(height=H, width=W, colorbar_position='bottom')
vfield_opts = opts.VectorField(width=W, height=H, color='Magnitude',
#                                magnitude=dim('Magnitude').norm()*0.2,
                               pivot='tip',
                               rescale_lengths=True)
curve_opts = opts.Points(size=5,width=W, height=H, padding=0.1, 
#                             xlim=(-10,10), ylim=(-10,10),
#                         color=dim('p')*256-50
                        )
contour_opts = opts.Contours(width=W, height=H, 
                             colorbar=False, 
                             tools=['hover'])

In [ ]:
# Grab registered bokeh renderer
print("Currently available renderers: ", *hv.Store.renderers.keys())
renderer = hv.renderer('bokeh')

In [ ]:
def UV2angMag(U,V):
    """
    Args:
    - U,V (MxN np.ndarray): encodes X,Y coordinate grids respectively
    
    Returns:
    - angle, mag: tuple of MxN np.ndarray that encode ang (or mag) for the grid space
    That means, angle[j][i] at (X[j][i],Y[j][i]) location
    """
    mag = np.sqrt(U**2 + V**2)
    angle = (np.pi/2.) - np.arctan2(U/mag, V/mag)

    return (angle, mag)

In [ ]:
test_h, test_w = 51, 51

test = np.zeros((test_h,test_w))
ones = np.ones((7,7))

test[7:14,7:14]=ones
test[27:34,27:34]=ones


plt.imshow(test, cmap='gray')

In [ ]:
kernel = np.array([[-0.5,0,0.5]])

gradx = correlate2d(test, kernel, mode='same')
grady = correlate2d(test, kernel.T, mode='same')

In [ ]:
f,ax = plt.subplots(1,2)
ax = ax.flatten()
ax[0].imshow(np.abs(gradx),cmap='gray');
ax[1].imshow(np.abs(grady),cmap='gray');


In [ ]:
test.shape, gradx.shape, grady.shape, kernel.shape


In [ ]:
# Explicitly specify the gird to link/ground the values to cartesian coord. system
test_xs, test_ys = list(range(test_w)), list(range(test_h))
# x,y coordinate values in grid
test_xx, test_yy = np.meshgrid(test_xs, test_ys)


In other words, we will explicitly say the coordinates and function evaluate at the coordinates
by providing actual <mark>values(!!)</mark> for the x,y coordinates! To visualize the numpy array (`test`) as shown, but grounded in our desired coordinates, we should flip the order of cooridnates in `test_yy` grid. Currently it's in decreasing order, but the cartesian cs's yaxis decreases as we move downwards.


In [ ]:
(
    hv.Image((test_xs, test_ys, test), label='ys=[0,..,50]').opts(img_opts) # test_ys = [0,...,50]
    + hv.Image((test_xs, np.flip(test_ys), test), label='ys=[50,..,0]').opts(img_opts)
).cols(2)

In [ ]:
test_angle, test_mag = UV2angMag(gradx,grady)


In [ ]:
gradfield = hv.VectorField( (test_xs, np.flip(test_ys), test_angle, test_mag) ).opts(vfield_opts)#

In [ ]:
gradfield

In summary, if you are visualizing a 2D data that is computed from the coordinate, eg: $zz = f(xx,yy) $, it is implicitly guaranteed that $ zz[j,i] == f(xx[j,i], yy[j,i])$. In this case, we don't need to worry about flipping the yaxis:

```python
xs = np.linspace(0,10)
ys = list(range(4))
xx, yy = np.meshgrid(xs, ys)
zz = xx+yy

hv.Image( (xs, ys, zz) )
```

However, if we need to specify which $zz[j,i]$ value corresponds to in x-y plane (eg. $zz$ is a numpy array or an image read from the disk), then we need to make sure the value $zz[j,i]$ is the value at $xx[j,i] and yy[j,i]$. In such case, we often need to flip `ys` to be in decreasing order (big->small) to be consistent with the Cartesian Coordinate System.

```python
img = imageio.imread('test.png')
xs = list(range(img.width))
ys = list(range(img.height)) # currently [0,...,height-1], ie. reverse of cartesian yaxis
ys = np.flip(ys) # now, consistent with the cartesiann yaxis order

hv.Image( (xs, ys, img) ) # this will show what you see with plt.imshow(img), grounded in the cartesian coordinate system

```

### Summary
Workflow to read image, compute its gradx, grady, and visualizing the gradient field using hv.VectorField


In [ ]:
# arr = iio.imread('../data/test/gradient.jpg')
# or create a test arr
arr = test

In [ ]:
# compute image gradients in x direction and y direction
kernel = np.array([[-1,1]]) #forward grad operation
# kernel = np.array([[-0.5,0, 0.5]]) #centered grad operation
gradx = correlate2d(test, kernel, mode='same')
grady = correlate2d(test, kernel.T, mode='same')
# angles and magnidute for the values at each [j,i] values in gradx and grady
ang,mag = UV2angMag(gradx, grady)

In [ ]:
# Since the `arr` is not grounded with x-y coordinates, we need to explicitly specify 
# which x,y coordinate arr[j,i] should be plotted on
nr, nc = arr.shape
xs = list(range(nc))
ys = list(range(nr)) # currently reverse order of Cartesian yaxis
ys = np.flip(ys) # now consistent with the Cartesian yaxis ordering

In [ ]:


(
    hv.Image( (xs, ys, arr) ).opts(img_opts)
    * hv.VectorField( (xs, ys, ang, mag) ).opts(vfield_opts)
)

Viola! now what we see using `holoviews` is exactly what we see using `plt.imshow`, and it is also what we intend to visualize. So, this shows how we can ground a numpy array (without x-y coordinate dimension labelled explicitly) to a cartesian coordinate system. 

The workflow is simpler, (ie. no need to flip the `ys`) when we can explicitly compute the `zz` value from `xx` and `yy`, as in that case, the mapping of `zz[j,i][ == func(xx[j,i], yy[j,i])` is gruanteed.


---

## 2. Visualizing `zz` computed from `xx` and `yy` 
Now, we are more interested in mathematical domain, ie. x,y coordinates -> function value at each coordinate, rather than visualizing an array which natually doesn't have a notion of a coordinate system

In [ ]:
def circle_phi(x,y):
    """Signed Distance function for a unit circle"""
    return np.sqrt(x**2 + y**2) - 1

def line_phi(x,y, ax,ay, bx, by):
    """
    Signed Distance function for a line with endpoints a,b 
    
    Args:
    - x,y (float): query point's x,y coordinate
    - ax, ay (float): one endpoint of the line
    - bx, by (float): the other endpoint of the line
    """
    return np.sqrt(x**2 + y**2) - 1

In [ ]:
n_points = 100
phi = circle_phi
xs = np.linspace(-2, 2, num=n_points)
ys = np.linspace(-1, 1, num=n_points)

xx,yy = np.meshgrid(xs, ys)
zz = phi(xx,yy)

zz_img = hv.Image( (xs, ys, zz) ).opts(img_opts).opts(colorbar_position='bottom')
zz_contour = hv.operation.contours(zz_img, levels=1).opts(contour_opts).opts(cmap='gray') #love this level=0:D


In [ ]:
gradx = correlate2d(zz, kernel, mode='same')
grady = correlate2d(zz, kernel.T, mode='same')
ang, mag = UV2angMag(gradx, grady)

In [ ]:
gradfield = hv.VectorField( (xs, ys, ang, mag) ).opts(vfield_opts)
gradfield

Finally, this is what we get:D

In [ ]:
(
    zz_img * zz_contour * gradfield
).opts(height=500, width=500)


In [ ]:
## sdf functions
from sdfs import *
# from vector import Vector as vec

In [ ]:
n_points = 100
xs = np.linspace(-2,2,n_points)
ys = np.linspace(-2,2,n_points)

In [ ]:
# wrapper to evaluate zz from sdf functions on the given x,y plane defined by `xs` and `ys`
def eval_sdf(xs, ys, sdFunc):
    zz = np.empty( (len(ys), len(xs)) )
    
    for j in range(len(ys)):
        for i in range(len(xs)):
            q = vec(xs[i],ys[j])
            zz[j,i] = sdFunc(q)
    return zz

In [ ]:
def unit_xline_sdf(query):
    a = vec(0.,0.)
    b = vec(1.,0.)
    return sdLine(query, a, b)


# Alternatively
from functools import partial
sdUnitHline = partial(sdLine, a=vec(0.,0.), b=vec(1.0, 0.0))
                      

## Let's try different sdfs 

In [ ]:
# zz = eval_sdf(xs, ys, partial(sdCircle, radius=1))
# sdfFunc = partial(sdTriangle, v0=vec(0,0), v1=vec(1,0), v2=vec(0.5, 0.5))
sdfFunc = partial(sdStar, radius=1, n=5, m=5.)

zz = eval_sdf(xs, ys, sdfFunc)

In [ ]:
zz_img = hv.Image( (xs, ys, zz) ).opts(img_opts) 
zz_contour = hv.operation.contours(zz_img, levels=0).opts(contour_opts).opts(cmap='gray') #love this level=0:D
zz_img * zz_contour;

In [ ]:
gradx = correlate2d(zz, kernel, mode='same')
grady = correlate2d(zz, kernel.T, mode='same')
ang, mag = UV2angMag(gradx, grady)

In [ ]:
gradfield = hv.VectorField( (xs, ys, ang, mag) ).opts(vfield_opts)
gradfield;

In [ ]:
(
    zz_img * zz_contour * gradfield
).opts(height=500, width=500)


In [ ]:
q = vec2(0.,-1.)
v0, v1, v2 = vec2(0,0), vec2(1,0), vec2(0.5, 0.5)
sdTriangle(q, v0, v1, v2)

In [ ]:
v = vec(1,0)
u = vec(0,1)
v.cross(u)

---
Modified: Jul 29, 2019
## Two ways to annotate hv.Image's axis
1. hv.Image((xs,ys,zz))
2. hv.Image(zz, bounds=(xmin,ymin,xmax,ymax)

## 1. hv.Image((xs,ys,zz))


In [ ]:
h,w = 10,10
xs = np.linspace(-1,1,num=w)
ys = np.linspace(-1,1,num=h)#[::-1]
zz = np.empty((w,h))
for i in range(len(xs)):
    for j in range(len(ys)):
        zz[j,i] = ys[j] 
plt.imshow(zz)

In [ ]:
%opts Image [tools=['hover'], colorbar_position='bottom'] (cmap='viridis')

In [ ]:
## zz is not carteisan aligned in itself. so to give the right semantic to its axis, ys must be 
## in the order when zz was computed from xs and ys values
(
    hv.Image((xs,ys,zz)) # does  work as expected
    + hv.Image((xs,ys[::-1], zz)) # doesn't work
    + hv.Image(zz, bounds=(-1,-1,1,1)) #doesn't work
)

On the other hand, if we make a contract to use `ys` as a yaxis, ie. its yvalues decrease as we iterate through the list (or equivalently, as its index increases), then all different cases are in conherent coordinate system, ie. the Cartesian system:

In [ ]:
h,w = 10,10
xs = np.linspace(-1,1,num=w)
ys = np.linspace(-1,1,num=h)[::-1]
zz = np.empty((w,h))
for i in range(len(xs)):
    for j in range(len(ys)):
        zz[j,i] = ys[j] 
        # underlying data itself is aligned with the cartesian coordinate system
plt.imshow(zz);
plt.title('Underlying data');

In [ ]:
(
    hv.Image((xs,ys,zz))
+ hv.Image(zz, bounds=(-1,-1,1,1))
)

### Another non-trivial example

In [ ]:
from sdfs import sdUnitHline, sdStar1,eval_sdf

xs = np.linspace(-1,1,10)
ys = np.linspace(-1,1,10)[::-1]
zz = eval_sdf(xs, ys, sdStar1)



In [ ]:
plt.imshow(zz)

In [ ]:
(
    hv.Image((xs,ys,zz)) + hv.Image(zz, bounds=(-1,-1,1,1))
)

## Conclusion
I will stick to the second convension from now on in order to avoid confusing myself. 
Here is the template I'm going to follow:


1. xs and ys are the sampled x and y axis in Cartesian cs

```python
xs = np.linspace(-1,1,w)
ys = np.linspace(-1,1,h)[::-1] 
```
2. Compute a function of the regular spaced xy sample domain 

```python
zz = zfunc(xs,ys) 
```
    At this point, zz's underlying data (as a `np.array` for instance) is in alignment with the Cartesian coordinate system.
    You can check with matplotlib:
    
```python
plt.imshow(zz)
```

3. hv.Image also gives the correctly displayed images:



In [ ]:
(
    hv.Image((xs,ys,zz))
+ hv.Image(zz, bounds=(-1,-1,1,1))
)

---
Modified: Jul 19, 2019
### `np.meshgrid` vs `np.mgrid`

In [ ]:
a = [1,2,3]
b = [10,20]
aa,bb = np.meshgrid(a,b)
nprint(aa,bb)

In [ ]:
zz = aa+bb

In [ ]:
hv.Image((a,b,zz)).opts(img_opts);

In [ ]:
np.mgrid[0:3, 0:2]

In [ ]:
a1, a2 = np.mgrid[1:4, 10:21:10]
nprint(a1, a1.T)
nprint (a2, a2.T)


In [ ]:
nprint(a2,a1)

### Summary 

In [ ]:
yy, xx = np.mgrid[10:21:10,1:4] #<-- order is: [yaxis slice, xaxis slice]
# gives the same result as
xx, yy = np.meshgrid( [1,2,3], [10,20] ) #<-- order is: (xval_list, yval_list)
# the idea is we can write quickly with mgrid without using `range(...)` or `np.linspace` alike.

- Use `np.mgrid` with ys, xs (with slice notation) as input to `np.mgrid[...]`. Note the square bracket. Then the output will be in order of `yy` and `xx`

- Use `np.meshgrid` with xs, ys specified with `range(...)` or `np.linspace` alike. 
    Then the output will be in order of `xx` and `yy`. 

Simple like that.